# Práctica 3 - Parte 1: Simulación de colas

## Objetivos
- Formular y simular un sistema de colas **M/M/c** (banco/aeropuerto) con **SimPy**.
- Estimar métricas clave: tiempos de espera, tiempos en sistema, utilización y rendimiento.
- **Comparar** resultados de simulación con las fórmulas **analíticas (Erlang C)**.
- Explorar **sensibilidad** al número de servidores y a la intensidad de llegadas.

_Instrucciones_: completa las celdas marcadas como ejercicios. Se recomienda estudiar la librería Simpy - https://simpy.readthedocs.io/en/latest/

## 📝 Enunciado
Modela un sistema de colas de un **aeropuerto o banco** con los siguientes supuestos (modelo **M/M/c**):
- Llegadas ~ Poisson con tasa $\lambda$ (tiempos entre llegadas $\sim \mathrm{Exp}(\lambda)$).  
- Tiempos de servicio $\sim \mathrm{Exp}(\mu)$.  
- $c$ servidores en paralelo, **cola única** y disciplina **FIFO**.  
- Capacidad de cola ilimitada. Horizonte de simulación: **8 horas**.

**Tareas**:
1. Implementa la llegada de clientes y el servicio con **SimPy**. Registra, para cada cliente, su **tiempo de espera** y **tiempo total** en el sistema - **CÓDIGO PROPUESTO**, explica cómo funciona.

2. Ejecuta **múltiples réplicas** y calcula los promedios: $\overline W_q$, $\overline W$, **utilización** y **throughput** (clientes/hora).

3. Calcula las métricas **analíticas** con **Erlang C** usando los mismos $\lambda,\mu,c$. Compara con la simulación.

4. Realiza un **análisis de sensibilidad** variando $c\in\{2,3,4\}$. Grafica $W_q$ (simulación vs. analítico) frente a $c$.


> Sugerencia de parámetros base: `SIM_TIME = 8*60` minutos, `c=3`, `mean_interarrival=1.8` (min), `mean_service=4.0` (min).


In [ ]:
# Preparación (instalar/importe de paquetes)
try:
    import simpy  # noqa: F401
except ImportError:
    !pip -q install simpy

import math, random, statistics
import numpy as np
import matplotlib.pyplot as plt

BASE_SEED = 12345
random.seed(BASE_SEED)
np.random.seed(BASE_SEED)


---
## ✍️ Ejercicio 1

1.   Implementación del modelo M/M/c: implementa la llegada de clientes y el servicio con **SimPy**. Registra, para cada cliente, su **tiempo de espera** y **tiempo total** en el sistema, además del resto de métricas descritas en la teoría - A continuación se muestra una **SOLUCIÓN PROPUESTA**.
2.   Describe el funcionamiento del código propuesto.



In [ ]:
# ================================
# Funciones auxiliares (Simulación)
# ================================

def expovariate(mean):
    """Devuelve una muestra exponencial con media 'mean'."""
    return random.expovariate(1.0/mean)

class Metrics:
    def __init__(self):
        self.wait_times = []      # esperas en cola
        self.system_times = []    # tiempos totales en sistema
        self.service_times = []   # tiempos de servicio
        self.arrivals = 0
        self.completed = 0
        self.total_service_time = 0.0  # suma de tiempos ocupados

def customer(env, name, counters, service_mean, metrics, sim_time):
    arrival_time = env.now
    metrics.arrivals += 1

    with counters.request() as req:
        yield req  # espera
        wait = env.now - arrival_time
        metrics.wait_times.append(wait)

        service = expovariate(service_mean)
        metrics.service_times.append(service)

        service_effective = service
        if env.now + service > sim_time:
            service_effective = max(0.0, sim_time - env.now)
        metrics.total_service_time += max(0.0, service_effective)

        yield env.timeout(service)
        system_time = env.now - arrival_time
        metrics.system_times.append(system_time)
        metrics.completed += 1

def arrival_process(env, interarrival_mean, counters, service_mean, metrics, sim_time):
    i = 0
    while True:
        ia = expovariate(interarrival_mean)
        if env.now + ia > sim_time:
            break
        yield env.timeout(ia)
        i += 1
        env.process(customer(env, f"cli_{i}", counters, service_mean, metrics, sim_time))

def run_mm_c(seed, interarrival_mean, service_mean, c, sim_time):
    import simpy
    random.seed(seed)
    env = simpy.Environment()
    counters = simpy.Resource(env, capacity=c)
    metrics = Metrics()

    env.process(arrival_process(env, interarrival_mean, counters, service_mean, metrics, sim_time))
    env.run(until=sim_time)

    avg_wait = statistics.mean(metrics.wait_times) if metrics.wait_times else 0.0
    avg_system = statistics.mean(metrics.system_times) if metrics.system_times else 0.0
    avg_service = statistics.mean(metrics.service_times) if metrics.service_times else 0.0
    utilization = metrics.total_service_time / (c * sim_time)

    return {
        "utilization": utilization, # rho
        "avg_wait": avg_wait, # Wq
        "avg_system": avg_system, # W
        "avg_service": avg_service,
        "arrivals": metrics.arrivals,
        "completed": metrics.completed
    }


---
## ✍️ Ejercicio 2:

Ejecuta **múltiples réplicas** y calcula los promedios: $ρ$, $\overline W_q$ y $\overline W$.

---
## ✍️ Ejercicio 3:

1.   Calcula las métricas **analíticas** con **Erlang C** usando los mismos $\lambda,\mu,c$ - **CÓDIGO PROPUESTO**
2.   Ejecuta la función en un nuevo bloque de código y compara con la simulación. ¿Tienen sentido los resultados para $ρ$, $\overline W_q$ y $\overline W$? Explícalo



In [ ]:
# ==================================
# Métricas analíticas (Erlang C)
# ==================================
def erlang_c_metrics(lmbda, mu, c):
    a = lmbda / mu               # carga ofrecida
    rho = lmbda / (c * mu)
    if rho >= 1.0:
        return {"stable": False, "rho": rho}

    sum_terms = sum((a**n) / math.factorial(n) for n in range(c))
    last = (a**c) / (math.factorial(c) * (1 - rho))
    P0 = 1.0 / (sum_terms + last)

    Pw = ((a**c) / (math.factorial(c) * (1 - rho))) * P0

    Wq = Pw / (c * mu - lmbda)
    W = Wq + 1/mu
    Lq = lmbda * Wq
    L = lmbda * W
    return {"stable": True, "rho": rho, "P0": P0, "Pw": Pw, "Wq": Wq, "W": W, "Lq": Lq, "L": L}

---
## ✍️ Ejercicio 4

Realiza un **análisis de sensibilidad** variando $c\in\{2,3,4\}$. Grafica $W_q$ (simulación vs. analítico) frente a $c$.
